In [1]:
import gym
import numpy as np

In [2]:
env=gym.make('FrozenLake-v0')

[2019-02-09 15:33:05,306] Making new env: FrozenLake-v0
/Users/falkvandermeirsch/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:18: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
env.render()

SFFF
FHFH
FFFH
HFFG



In [4]:
a = np.zeros((1, env.observation_space.n))
b = np.ones((1, env.observation_space.n))
print(a.shape)
print(b.shape)
c = np.concatenate((a, b), axis=0)
print(c.shape)
np.concatenate((c,a), axis=0)

(1, 16)
(1, 16)
(2, 16)


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
def compute_value_function(env, policy, gamma):
    # initialize value table
    value_table = np.zeros((1, env.observation_space.n))
    value_history = value_table
    policy_history = np.expand_dims(policy, axis=0)
    
    # set a treshold
    threshold = 1e-20
    
    while True:
        # copy the value table
        updated_value_table = np.copy(value_table)
        
        # for each state
        
        for state in range(env.observation_space.n):
            # select action from the policy
            action = policy[state]
            
            # update the value table
            value_table[0][state] = sum(
                (trans_prob * (reward_prob + gamma * updated_value_table[0][next_state])) 
                for trans_prob, next_state, reward_prob, _ in env.P[state][action])
        
        value_history = np.concatenate((value_history, value_table), axis=0)
        policy_history = np.concatenate((policy_history, np.expand_dims(policy, axis=0)), axis=0)
        
        if np.sum(np.abs(updated_value_table[0] - value_table[0])) <= threshold:
            break
    
    return value_table[0], value_history, policy_history
            

In [8]:
compute_value_function(env, np.zeros(env.observation_space.n), gamma=0.9)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [9]:
def extract_policy(env, value_table, gamma=1.0):
    # initializing the policy table
    policy=np.zeros(env.observation_space.n)

    for state in range(env.observation_space.n):

        # initializing the Q_table for each state
        Q_table=np.zeros(env.action_space.n)

        # compute Q Value for each actions of the state
        #Q_value(state,action) <-sum over s' (p(s'|s,a)*(r(s'|s,a)+gamma*V(s')))
        for action in range(env.action_space.n):
            for next_s in env.P[state][action]:
                trans_prob, next_state, reward_prob, _=next_s
                # adding the next_state_rewards for the succesor state next_state to the Q_value
                Q_table[action]+=(trans_prob*(reward_prob+gamma*value_table[next_state]))

        #select the action that has the maximum Q-value for finding the optimal policy
        policy[state]=np.argmax(Q_table)
        #pi(state)<-argmax over actions (Q_table(state, action))

    return policy


In [10]:
policy_history = np.zeros((4,4))
value_history = np.zeros((4,4))

In [11]:
def policy_iteration(env, gamma):
    # initialize policy
    old_policy = np.zeros(env.observation_space.n)
    n_iterations = 100_000
    
    history_values = np.zeros((1, env.observation_space.n))
    history_policies = np.zeros((1, env.observation_space.n))
    
    for i in range(n_iterations):
        # compute value function of the current policy
        new_value_function, val_hist, pol_hist = compute_value_function(env, old_policy, gamma)
        history_values = np.concatenate((history_values, val_hist), axis=0)
        history_policies = np.concatenate((history_policies, pol_hist), axis=0)
        
        # extract old policy - policy improvement step
        new_policy = extract_policy(env, new_value_function, gamma)
        
        # check whether we have reached convergence
        if np.all(old_policy == new_policy):
            print(f'Policy Iteration converged at step {i+1}')
            break
        
        old_policy = new_policy
    
    return new_policy, history_values, history_policies
        

In [16]:
policy, value_history, policies_history = policy_iteration(env, gamma=0.9)

Policy Iteration converged at step 6


In [23]:
value_history.shape
policies_history[500]

array([0., 3., 2., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])

In [48]:
d = {
    0: "←",
    1: "↓",
    2: "→",
    3: "↑",
}

arrows = np.chararray(policies_history.shape, unicode=True)
for i, policy in enumerate(policies_history):
    for j, action in enumerate(policy):
        arrows[i][j] = d[action]


In [49]:
from ipywidgets import interact
import matplotlib.pyplot as plt
from itertools import product

@interact
def view_image(i= (0,1000)):
    plt.clf()
    x = range(4)
    y = range(4)
    for a,b in product(x, y): 
        plt.text(a, b, f'{np.transpose(arrows[i].reshape(4,4))[a][b]}',)
    plt.imshow(value_history[i].reshape(4,4), cmap='winter')
    plt.show()

interactive(children=(IntSlider(value=500, description='i', max=1000), Output()), _dom_classes=('widget-intera…